In [1]:
import pandas as pd
import geopandas as gdp
import keplergl

## Leer los datos de terremotos

In [2]:
# read csv
df = pd.read_csv('data/Worldwide-Earthquake-database.csv')
df

,I_D,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,FOCAL_DEPTH,EQ_PRIMARY,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
0,1,No,-2150,NaN,NaN,NaN,NaN,,NaN,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Yes,-2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,No,-2000,NaN,NaN,NaN,NaN,NaN,18.0,7.1,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
3,5877,Yes,-1610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
4,8,No,-1566,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,10488,No,2020,4.0,1.0,12.0,23.0,27,10.0,5.3,...,NaN,NaN,NaN,NaN,11.28,3.0,NaN,NaN,790.0,3.0
6189,10490,Yes,2020,5.0,2.0,12.0,51.0,6,17.0,6.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6190,10491,No,2020,5.0,7.0,20.0,18.0,22,10.0,4.6,...,NaN,NaN,22.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6191,10494,No,2020,5.0,18.0,13.0,48.0,3,10.0,5.2,...,NaN,NaN,24.0,1.0,NaN,2.0,NaN,1.0,NaN,NaN


## Limpieza y preparación de datos

Convertimos las columnas de latitud y longitud a numéricas, los errores se convertirán en NaN

In [4]:
df['LONGITUDE'] = pd.to_numeric(df.LONGITUDE, errors='coerce')
df['LATITUDE'] = pd.to_numeric(df.LATITUDE, errors='coerce')

Nos deshacemos de las filas con datos perdidos para latitud, longitud e intensidad (del terremoto)

In [5]:
df.dropna(subset=['LONGITUDE', 'LATITUDE', 'INTENSITY'], inplace=True)

También convertimos la columna "FLAG_TSUNAMI" a números enteros, la necesitaremos en ese formato

In [6]:
df['FLAG_TSUNAMI'] = [1 if i=='Yes' else 0 for i in df.FLAG_TSUNAMI.values]

In [9]:
df

,I_D,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,FOCAL_DEPTH,EQ_PRIMARY,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
1,2,1,-2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0,-2000,NaN,NaN,NaN,NaN,NaN,18.0,7.1,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
4,8,0,-1566,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,11,0,-1450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,14,0,-759,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6184,10483,0,2020,3.0,10.0,10.0,18.0,6,23.0,5.0,...,NaN,NaN,4.0,1.0,NaN,2.0,NaN,NaN,1137.0,4.0
6185,10485,0,2020,3.0,18.0,13.0,9.0,31,12.0,5.7,...,NaN,NaN,NaN,NaN,48.5,4.0,NaN,NaN,NaN,3.0
6186,10486,0,2020,3.0,22.0,5.0,24.0,3,10.0,5.4,...,NaN,NaN,27.0,1.0,6000.0,4.0,6305.0,4.0,20000.0,4.0
6187,10487,1,2020,3.0,25.0,2.0,49.0,21,57.0,7.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Las columnas de latitud y longitud no se veían en la muestra anterior. Las mostramos en esta ocasión para corroborar su formato.

In [10]:
df.loc[:,('LATITUDE','LONGITUDE')]

,LATITUDE,LONGITUDE
1,35.683,35.800
2,38.000,58.200
4,31.500,35.300
5,35.500,25.500
9,33.000,35.500
...,...,...
6184,-6.808,106.676
6185,40.751,-112.078
6186,45.897,15.966
6187,48.986,157.693


## Generamos un mapa y le agregamos datos

Podemos desplegar un menú del lado izquierdo, contiene lo siguiente:
- Layers — Define cómo es que las variables son agregadas al mapa (recuerda que una variable es una columna en tus datos).
- Filters — Desde aquí podemos filtrar para obtener muestras más pequeñas de nuestro conjunto de datos.
- Interactions — Define las interacciones, como los tooltips, las cajas de leyendas, entre otras cosillas.
- Basemap — Define el estilo de los elementos en el mapa cDefines the style of the world map and other elements like labels, roads, styles

Nota de Omar del pasado: Omar del futuro haz lo siguiente.
- Agrega una nueva capa, utiliza hexágonos y elige el radio apropiado

In [11]:
kepler_map = keplergl.KeplerGl(height=400)
kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl()

In [12]:
kepler_map.add_data(data=df, name="earthquakes")